# Distance between CoCs
A factory can be placed in a CoC and serves two functions:

1. Provide shelter to people experiencing homelessness (PEH) in that CoC.
2. Serve as a source for other temporary housing units (THUs) that can be deployed to other CoCs.

To determine the optimal deployment of factories, we require the distance between each pair of CoCs; from this distance we can calculate a transportation cost that will be used in the optimization framework to strategically deploy THUs and factories. In this work, we consider two ways to calculate the distance between CoCs:

1. Haversine distance
2. Road network distance

## Haversine distance
The hversine distance determines the orthodromic distance between two points on a sphere. Given the latitude $\phi$ and longitude $\lambda$ of two points, we can calculate the distance according to:
$$
d = 2r \sin^{-1}\left( \sqrt \frac{1 - \cos(\Delta \phi) + \cos(\phi_1) \cos(\phi_2)(1-\cos(\Delta \lambda))}{2} \right)
$$
where $r$ is the radius of the Earth (approximately 6,370 km).

In [109]:
import math
# Earth radius R in miles
R = 3958.8
def haversine_dist(lat1, lon1, lat2, lon2):
    # Convert to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    delta_latitude = lat2_rad - lat1_rad
    delta_longitude = lon2_rad - lon1_rad
    # sine version    
    return 2*R*math.asin(math.sqrt((1 - math.cos(delta_latitude) + math.cos(lat1_rad)*math.cos(lat2_rad)*(1 - math.cos(delta_longitude)))/2))

    # arctan version
    # a = (1 - math.cos(delta_latitude) + math.cos(lat1_rad)*math.cos(lat2_rad)*(1 - math.cos(delta_longitude)))/2
    # return 2*R*math.atan2(math.sqrt(a), math.sqrt(1-a))

print(haversine_dist(33.5, -86.67, 30.76, -87.93))

203.16073789703498


In [76]:
import geopandas as gpd
import os
from pathlib import Path

state_abbreviations = [
    "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA",
    "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD",
    "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ",
    "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC",
    "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"
]

states = [
    'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 
    'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 
    'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 
    'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 
    'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 
    'New_Hampshire', 'New_Jersey', 'New_Mexico', 'New_York', 
    'North_Carolina', 'North_Dakota', 'Ohio', 'Oklahoma', 'Oregon', 
    'Pennsylvania', 'Rhode_Island', 'South_Carolina', 'South_Dakota', 
    'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 
    'West_Virginia', 'Wisconsin', 'Wyoming'
]


abbreviation_state_pairs = list(zip(state_abbreviations, states))
for i in range(len(abbreviation_state_pairs)):
    print(abbreviation_state_pairs[i])


('AL', 'Alabama')
('AK', 'Alaska')
('AZ', 'Arizona')
('AR', 'Arkansas')
('CA', 'California')
('CO', 'Colorado')
('CT', 'Connecticut')
('DE', 'Delaware')
('FL', 'Florida')
('GA', 'Georgia')
('HI', 'Hawaii')
('ID', 'Idaho')
('IL', 'Illinois')
('IN', 'Indiana')
('IA', 'Iowa')
('KS', 'Kansas')
('KY', 'Kentucky')
('LA', 'Louisiana')
('ME', 'Maine')
('MD', 'Maryland')
('MA', 'Massachusetts')
('MI', 'Michigan')
('MN', 'Minnesota')
('MS', 'Mississippi')
('MO', 'Missouri')
('MT', 'Montana')
('NE', 'Nebraska')
('NV', 'Nevada')
('NH', 'New_Hampshire')
('NJ', 'New_Jersey')
('NM', 'New_Mexico')
('NY', 'New_York')
('NC', 'North_Carolina')
('ND', 'North_Dakota')
('OH', 'Ohio')
('OK', 'Oklahoma')
('OR', 'Oregon')
('PA', 'Pennsylvania')
('RI', 'Rhode_Island')
('SC', 'South_Carolina')
('SD', 'South_Dakota')
('TN', 'Tennessee')
('TX', 'Texas')
('UT', 'Utah')
('VT', 'Vermont')
('VA', 'Virginia')
('WA', 'Washington')
('WV', 'West_Virginia')
('WI', 'Wisconsin')
('WY', 'Wyoming')


In [ ]:
# # Sample processing of .shp
# current_dir = os.getcwd()
# gdf = gpd.read_file(current_dir + "\CoC_GIS_State_Shapefile_MA\Massachusetts\MA_500\MA_500.shp")
# # gdf.info()

# # print(len(gdf))

# # Compute centroids
# centroids = gdf.geometry.centroid

# # Extract latitudes and longitudes
# longitudes = centroids.x
# latitudes = centroids.y

# # Compute averages
# avg_lon = longitudes.mean()
# avg_lat = latitudes.mean()

# print("Latitude, longitude: ", avg_lat, avg_lon)

# # # Get the centroid of the first geometry
# # centroid = gdf.geometry.iloc[0].centroid

# # # Get latitude and longitude
# # longitude = centroid.x
# # latitude = centroid.y

# # print("Latitude, longitude: ", latitude, longitude)


<class 'numpy.float64'>
Latitude, longitude:  42.315586871377704 -71.09145995733913


<>:3: SyntaxWarning: invalid escape sequence '\C'
<>:3: SyntaxWarning: invalid escape sequence '\C'
C:\Users\yanna\AppData\Local\Temp\ipykernel_13620\2766677258.py:3: SyntaxWarning: invalid escape sequence '\C'
  gdf = gpd.read_file(current_dir + "\CoC_GIS_State_Shapefile_MA\Massachusetts\MA_500\MA_500.shp")


In [ ]:
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

# Get CoCs for all states
cocs = []
coc_lon_lat = {}
directory = Path.cwd().parent
for abbreviation, state in abbreviation_state_pairs:
    # Get CoC paths and CoCs per state folder
    if state == "Wyoming":
        coc_path = directory / 'homelessness-prediction' / 'data/coc-shapefiles/2024' / f'CoC_GIS_State_Shapefile_{abbreviation}'
    else:
        coc_path = directory / 'homelessness-prediction' / 'data/coc-shapefiles/2024' / f'CoC_GIS_State_Shapefile_{abbreviation}' / state
    for coc in os.listdir(coc_path):
        if coc.startswith(abbreviation + '_'):
            cocs.append(coc)
            gdf = gpd.read_file(coc_path / coc / f'{coc}.shp')
            # Compute centroids (if more than one)
            centroids = gdf.geometry.centroid

            # Extract latitudes and longitudes (if more than one)
            longitudes= centroids.x
            latitudes = centroids.y

            # Compute average of centroids (if more than one)
            avg_lon = longitudes.mean()
            avg_lat = latitudes.mean()

            coc_lon_lat[coc] = (avg_lon, avg_lat)

numCoCs = len(cocs)

# Validation
# print(cocs)
# print(numCoCs)
# print(coc_lon_lat)


['AL_500', 'AL_501', 'AL_502', 'AL_503', 'AL_504', 'AL_505', 'AL_506', 'AL_507', 'AL_508', 'AK_500', 'AK_501', 'AZ_500', 'AZ_501', 'AZ_502', 'AR_500', 'AR_501', 'AR_503', 'AR_505', 'AR_508', 'CA_500', 'CA_501', 'CA_502', 'CA_503', 'CA_504', 'CA_505', 'CA_506', 'CA_507', 'CA_508', 'CA_509', 'CA_510', 'CA_511', 'CA_512', 'CA_513', 'CA_514', 'CA_515', 'CA_516', 'CA_517', 'CA_518', 'CA_519', 'CA_520', 'CA_521', 'CA_522', 'CA_523', 'CA_524', 'CA_525', 'CA_526', 'CA_527', 'CA_529', 'CA_530', 'CA_531', 'CA_600', 'CA_601', 'CA_602', 'CA_603', 'CA_604', 'CA_606', 'CA_607', 'CA_608', 'CA_609', 'CA_611', 'CA_612', 'CA_613', 'CA_614', 'CO_500', 'CO_503', 'CO_504', 'CO_505', 'CT_503', 'CT_505', 'DE_500', 'FL_500', 'FL_501', 'FL_502', 'FL_503', 'FL_504', 'FL_505', 'FL_506', 'FL_507', 'FL_508', 'FL_509', 'FL_510', 'FL_511', 'FL_512', 'FL_513', 'FL_514', 'FL_515', 'FL_517', 'FL_518', 'FL_519', 'FL_520', 'FL_600', 'FL_601', 'FL_602', 'FL_603', 'FL_604', 'FL_605', 'FL_606', 'GA_500', 'GA_501', 'GA_502',

In [ ]:
# Produce distance matrix
# Create city distances matrix
import pandas as pd
import numpy as np
coc_distance_matrix = pd.DataFrame(columns=[f'{coc}' for coc in cocs])

for coc_source, lon_lat_source in coc_lon_lat.items():
    coc_distance_arr = []
    for coc_dest, lon_lat_dest in coc_lon_lat.items():
        distance = haversine_dist(lat1=lon_lat_source[1], lon1=lon_lat_source[0], lat2=lon_lat_dest[1], lon2=lon_lat_dest[0])
        # print(coc_source)
        # print(coc_dest)
        # print(lon_lat_source)
        # print(lon_lat_dest)
        # print(distance)
        coc_distance_arr.append(distance)
    coc_distance_matrix.loc[len(coc_distance_matrix)] = coc_distance_arr
    # if coc_source == "AL_503":
    #     break

# print(coc_distance_matrix.head())

coc_distance_matrix['CoC'] = cocs
coc_distance_matrix.set_index('CoC', inplace=True)

# Validation 
# print(coc_distance_matrix.shape)
# print(coc_distance_matrix.head())
# print(coc_distance_matrix['AL_504']['AL_505'])
# print(coc_distance_matrix.iloc[4,5]) # should be equivalent to previous
# print(coc_lon_lat['AL_504'])
# print(coc_lon_lat['AL_505'])




128.93165333114396
128.93165333114396
(-86.27754107332348, 32.313066637035206)
(-85.81181183160722, 34.13798188935623)
